In [1]:
#IMPORT DE BIBLIOTECAS

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

from matplotlib import pyplot as plt
import seaborn as sns

from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy
from surprise.model_selection import cross_validate

from sklearn.decomposition import TruncatedSVD
import os

In [2]:
df_de_cat = pd.read_csv('category_de.txt') # DataFrame Informativo das categorias

In [3]:
clicks_de = pd.read_csv('clicks_de_sample_2.txt', sep = ',', header=0) # df de clicks

In [4]:
# CRIAÇÃO DE COLUNA DE CLICKS POR CATEGORIA
clicks_de.drop(columns = ['Keywords'], axis = 1, inplace=True)
#clicks_de['Category'] = clicks_de['Category'].astype(str)

pontuacao_de = pd.DataFrame(clicks_de['Category'].value_counts()).reset_index()
pontuacao_de.columns = ['Category', 'Pontuacao']
dici = dict(zip(pontuacao_de.Category, pontuacao_de.Pontuacao))

clicks_de['Cat_clicks'] = clicks_de['Category'].map(dici)
clicks_de.sort_values(by='Cat_clicks', inplace = True)

In [5]:
## RENOMEANDO OFERTAS E USUÁRIOS
clicks_de['User'] = clicks_de['UserId'].map(dict(zip(clicks_de['UserId'].unique(), 
                                                                        range(len(clicks_de['UserId']))))) # Redefinindo indices para usuário apenas como numérico

clicks_de['Offer'] = clicks_de['OfferTitle'].map(dict(zip(clicks_de['OfferTitle'].unique(), 
                                                                        range(len(clicks_de['OfferTitle']))))) #  Redefinindo indices para ofertas apenas como numérico considerando o OfferTitle

In [6]:
clicks_de = clicks_de.merge(df_de_cat, left_on = 'Category', right_on = 'ID').drop('ID', axis = 1) 

In [7]:
clicks_per_user_product = clicks_de.groupby(by=['UserId','OfferId']).count()[['Name']].reset_index()

In [8]:
clicks_per_user_product['User'] = clicks_per_user_product['UserId'].map(dict(zip(clicks_per_user_product['UserId'].unique(), 
                                                                        range(len(clicks_per_user_product['UserId']))))) # Redefinindo indices para usuário apenas como numérico


clicks_per_user_product['Offer'] = clicks_per_user_product['OfferId'].map(dict(zip(clicks_per_user_product['OfferId'].unique(), 
                                                                        range(len(clicks_per_user_product['OfferId']))))) #  Redefinindo indices para ofertas apenas como numérico considerando o OfferTitle

In [9]:
clicks_de['UserClicks'] = clicks_de['UserId'].map(dict(zip(clicks_de['UserId'].value_counts().index,
                                                           clicks_de['UserId'].value_counts().values))) #Criação de coluna com nº total de cliques do usuário

In [10]:
#Cap minimo de clicks para integrar o sistema de recomendação
clicks_de_filtered = clicks_de[(clicks_de.UserClicks > 40)]

#DataFrame de base para utilização no Surprise
user_item_clicks = clicks_de_filtered[['User','Offer','UserClicks']]

In [11]:
log = user_item_clicks.groupby(by=['User','Offer']).count().reset_index()
log['UserClicks'] = 1
display(log.head())

,User,Offer,UserClicks
0,249,83,1
1,249,3481,1
2,249,7628,1
3,249,7708,1
4,249,9575,1


In [12]:
#Matriz esparsa
pivo_log = log.pivot_table(columns = 'Offer', index = 'User', values = 'UserClicks').fillna(0)

In [13]:
#Nova relação de clicks em cada produto por usuário a partir da matriz esparsa para uso no Surprise
pivo_log_melted = pd.melt(frame = pivo_log, var_name = 'Products', value_name = 'Clicked?', ignore_index = False).reset_index()

## TESTES SURPRISE

In [15]:
#Escala de avaliação
reader = Reader(rating_scale = (0,1))

#Criação de objeto Dataset do Surprise
data = Dataset.load_from_df(df = pivo_log_melted, reader=reader)

In [16]:
#Criação do modelo e teste por validação cruzada
svd = SVD() # Falta otimização de parametros e testagem de outros modelos
cross_validate(svd, data, measures = ['RMSE','MAE'], cv = 3)

{'test_rmse': array([0.13086859, 0.13328266, 0.13356066]),
 'test_mae': array([0.03479759, 0.03536657, 0.03674502]),
 'fit_time': (8.72905683517456, 8.114208698272705, 8.654720544815063),
 'test_time': (0.6122486591339111, 0.6173496246337891, 0.6891317367553711)}

In [17]:
#Criação do set de treino com o dataset completo
trainsetfull = data.build_full_trainset()

#Fit do modelo com o set de treino
svd.fit(trainsetfull)

In [ ]:
#trainset_iids = list(trainset.all_items())
#iid_converter = lambda x: trainset.to_raw_iid(x)
#trainset_raw_iids = list(map(iid_converter, trainset_iids))

In [ ]:
#Fetch de informação de usuários pra teste
pivo_log_melted[(pivo_log_melted['User']==815) & (pivo_log_melted['Clicked?']==1)]

In [60]:
#Teste de predição com usuário específico frente a um produto específico
svd.predict(uid = 815, iid = 282283)

Prediction(uid=815, iid=282283, r_ui=None, est=0.17134022028105783, details={'was_impossible': False})

# TO DO:
* SIMILARIDADE DE USUARIOS E ITENS
* TUNAGEM DE HIPERPARAMETROS
* LISTAS DE RECOMENDAÇÕES
* DEPLOY
